**external memory model**

**Зачем?**

Оперативная память дорогая. К тому же при отключении питания на ней информация не сохраняется, в отличие от жестких дисков. Но на жестких дисках в среднем 7200 оборотов в минуту - это порядка 100 чтений в секунду, то есть **10ms** (RAM:  **~ 100 ns**, то есть в 100 000 раз быстрее) на запрос. В то время как скорость передачи данных порядка 100 МБайт/сек, то есть **$10^9$** бит в секунду. *Так что хочется минимизировать поиск и максимизировать объем загрузки после каждого чтения*

Чтобы скомпенсировать время поиска и чтения, мы будем считывать сразу $B$ информации (она порядка МБайта)

**B-Search**

$B$- поиск вместо бинарного через статичное $B-Tree$ за время $\mathcal{O}(\log_{B}n)$. Можно показать, что **быстрее нельзя:** у нас после каждого прочтения $B$ бит возможных $B+1$ вердикта (искомый $x$ левее всех, 1й слева, 2й и т.д.). Рисуем дерево решений и получаем ответ.

**B-Merge**

При обычном merge мы не хотели делать сразу несколько потоков, потому что там тяжело вычислять минимум. В алгоритмах внешней памяти мы себе это можем позволить и реализовать слияние нескольких потоков с помощью приоритетной очереди.

Будем делить массив на $\frac{M}{B}$ потоков с буфером размера $B$, складывая элементы в приоритетную очередь.
Разбиваем на кусочки до тех пор, пока массив не поместится целиком в память. То есть в листьях дерева ветвлений порядка $M$ элементов, а разветвление происходит на $\frac{M}{B}$ детей. На каждом уровне слияние происходит за порядка $\frac{n}{B}$ чтений. Как внутри памяти это реализовано (тупым проходом каждый раз, либо же через кучу - это мы **не учитываем**). Суммарно $\mathcal{O}\left(\frac{n}{B}\log_{\frac{M}{B}}\frac{n}{M}\right)$, на практике глубина порядка 1-2, почти всё считается в памяти после первого же деления.

**cache oblivious model**

Суть в том, что мы теперь рассматриваем кэш как внешнюю память, но при этом нам неизвестны ничьи размеры. 
Это интересно из-за того, что мы не можем управлять на практике работой кэша, мы лишь делаем запросы.

Обычный merge sort работает за $\mathcal{O}(\frac{n}{B}\log \frac{n}{M})$, даже несмотря на то, что мы не знаем ни $M$, ни $B$.

Но есть нумерация **Ван Эмде Боаса**. 
* Она строится рекурсивно (и корректное построение я видел только для размеров $2^{2^k}$, с постоянным делением высоты пополам и переходом к $n\rightarrow \sqrt{n}$. 
* В таком случае, каким бы ни было $B$, в какой-то момент рассмотренная нами рекурсивная процедура нумерации станет помещаться целиком внутрь $B$ и мы будем за одно считывание проходить порядка $\log B$ высоты, при этом всё дерево порядка $\log n$. 
* Итого **cache miss'ов $\log_B n$**